In [1]:
import open3d as o3d
import numpy as np
import copy

import time

In [2]:
# transforem the source cloud and then draw the point clouds
def draw_registration_result(source, target, transformation):
    #copy original point clouds
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #set the color
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    #apply transformation
    source_temp.transform(transformation)
    #draw the clouds
    o3d.visualization.draw_geometries([source_temp,target_temp])
    #o3d.visualization.draw_geometries([target])

In [3]:
#down sample the point cloud, recompute new normals, compute fpfh features
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    #use the neighbors recompute the normal
    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    #compute the FPFH features
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh



In [4]:
#load data, down sample and compute fpfh feature
def prepare_dataset(voxel_size):
    #load point clouds
    print(":: Load two point clouds and disturb initial pose.")
    source = o3d.io.read_point_cloud("/home/yunke/3dv_proj/code/badslam0427/new_bg/bg.ply")
    target = o3d.io.read_point_cloud("/home/yunke/3dv_proj/code/badslam0427/cereal_g/lie/lie.ply")
    
    #apply an inital transformation
    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    #visualize
    draw_registration_result(source, target, np.identity(4))

    #downsample and compute fpfh feature of the point clouds
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [5]:
#global registration based on RANSAC
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    
    #apply RANSAC algorithms for registration:
    #pick 4 points,
    #pruning with edge length and distance for early stop
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

In [6]:
#fast global registration
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [7]:
#point to plane registration
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.registration.registration_icp(
        source, target, distance_threshold, result_fast.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    return result

In [8]:
#crop point cloud+outlier
def crop_pcd_target(pcd,source,result_icp):
    source_temp = copy.deepcopy(source)
    cor_ind = np.asarray(result_icp.correspondence_set)
    pcd_out = pcd.select_down_sample(cor_ind[:,1],invert=True)
    out_dist = np.asarray(pcd_out.compute_point_cloud_distance(source_temp.transform(result_icp.transformation)))
    out_index = np.arange(0,out_dist.shape[0],1)
    #threshold
    thr = 0.01
    #select outliers
    out_list = out_index[out_dist>thr]
    pcd_out = pcd_out.select_down_sample(out_list)
    #define bounding box
    center = np.asarray([[0.0,-0.1,0.60]]).reshape((3,1))
    R = np.eye(3)
    extent = np.asarray([[1.0,1.0,1.0]]).reshape((3,1))
    box = o3d.geometry.OrientedBoundingBox(center,R,extent)
    print(box)
    #crop
    cropped = pcd_out.crop(box)
    
    return cropped

In [9]:
#crop point cloud+outlier
def crop_pcd_source(pcd):
    
    #define bounding box
    
    center = np.asarray([[0.0,-0.1,0.60]]).reshape((3,1))
    R = np.eye(3)
    extent = np.asarray([[1.0,1.0,1.0]]).reshape((3,1))
    box = o3d.geometry.OrientedBoundingBox(center,R,extent)
    print(box)
    #crop
    cropped = pcd.crop(box)
    
    return cropped

In [10]:
#segment inlier plane
def remove_plane(target,source,result_icp):
    #crop
    target_crop = crop_pcd_target(target,source,result_icp)
    o3d.visualization.draw_geometries_with_editing([target_crop])
    source_temp = copy.deepcopy(source)
    source_temp.transform(result_icp.transformation)
    source_crop = crop_pcd_source(source_temp)
    o3d.visualization.draw_geometries_with_editing([source_crop])
    #combine point clouds, make the plane to be removed easily
    tgt_src = target_crop + source_crop + source_crop
    o3d.visualization.draw_geometries_with_editing([tgt_src])
    #segement 
    plane,points_on_plane = tgt_src.segment_plane(0.015,4,1000)
    #select not on plane only from target point cloud
    full_ind = np.arange(np.asarray(target_crop.points).shape[0])
    not_on_plane_ind = np.setdiff1d(full_ind,points_on_plane)
    return tgt_src.select_down_sample(not_on_plane_ind)

In [11]:
#cluster
def cluster_points(target_no_plane):
    full_index = np.arange(np.asarray(target_no_plane.points).shape[0])

    #cluster
    cluster_class = np.asarray(target_no_plane.cluster_dbscan(voxel_size*0.25, 20, print_progress=False))
    #select cluster
    min_dist = 1.0
    obj_points = copy.deepcopy(target_no_plane)
    for cluster in range(max(cluster_class)+1):
        
        cur_cluster_ind = full_index[cluster_class==cluster]
        cur_cluster_points = target_no_plane.select_down_sample(cur_cluster_ind)
        #compute center
        cur_points = np.asarray(cur_cluster_points.points)
        cur_mean = np.mean(cur_points,axis=0)
        print(cur_mean)
        cur_dist = np.linalg.norm(cur_mean-np.asarray([[0.0,0.0,0.6]]))
        print(cur_dist)
        if cur_dist<min_dist and cur_points.shape[0]>1000:
            min_dist = cur_dist
            obj_points = cur_cluster_points
            
    return obj_points
            

In [12]:
#get object
def get_object_point_cloud(target,source,result_icp):
    #remove plane
    target_no_plane = remove_plane(target,source,result_icp)
    #cluster points
    obj_points = cluster_points(target_no_plane)
    o3d.visualization.draw_geometries([obj_points])
    return obj_points

In [13]:
voxel_size = 0.05  # means 5cm for the dataset

#down sample, compute fpfh of the data
source, target, source_down, target_down, source_fpfh, target_fpfh = \
        prepare_dataset(voxel_size)

#global registration using ransac
start = time.time()
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print("Global registration took %.3f sec.\n" % (time.time() - start))
print(result_ransac)


# #draw the result
# draw_registration_result(source_down, target_down,
#                         result_ransac.transformation)

#fast global registration
start = time.time()
result_fast=execute_fast_global_registration(source_down, target_down,
                                             source_fpfh, target_fpfh,
                                             voxel_size)
                         
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
# draw_registration_result(source_down, target_down,
#                         result_fast.transformation)
    
#local refinement
result_icp = refine_registration(source, target, source_fpfh, target_fpfh,
                                 voxel_size)
# print(result_icp)
# draw_registration_result(source, target, result_icp.transformation)

obj_points = get_object_point_cloud(target,source,result_icp)
#save
o3d.io.write_point_cloud("objects/creal_g_lie.pcd",obj_points)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
Global registration took 1.762 sec.

registration::RegistrationResult with fitness=9.656524e-01, inlier_rmse=3.147987e-02, and correspondence_set size of 10346
Access transformation to get result.
:: Apply fast global registration with distance threshold 0.025
Fast global registration took 0.365 sec.

registration::RegistrationResult with fitness=6.747247e-01, inlier_rmse=1.786993e-02, and correspondence_set size of 7229
Access transformation to get result.
:: Point-to-plane ICP registration is applied on original point
   clouds 

True